### Moving Power BI Semantic Models in Large Semantic Model Storage Format Across Regions
#### Background and Purpose
Power BI semantic models (formerly known as "datasets") that use the "Large Semantic Model" storage format can not be moved across regions (as documented in the following article: https://learn.microsoft.com/en-us/fabric/admin/service-admin-premium-multi-geo?tabs=power-bi-premium). If a workspace containing a semantic model in large storage format is assigned to a capacity in a different region, the semantic model will become unusable (i.e., reports associated this semantic model will fail to load).

This notebook is intended to help you identify semantic models that use the large semantic model storage format across your Fabric tenant when you are considering moving certain workspaces to capacities in different regions. If you have such semantic models, remediation suggestions provided at the bottom of the notebook will help you understand potential options for completing the move.

#### Usage Instructions
##### Prerequisites
1. This notebook is intended to be hosted and run in a workspace hosted on a Fabric/Premium capacity in your Fabric tenant.
1. This notebook uses the Semantic Link library which assists with the authentication to Power BI Rest APIs using the identity of the user running this notebook.
1. This notebook uses Power BI Admin APIs and requires the user running this notebook to be a member of the Fabric Admin role.

##### Running the Notebook
1. If desired, specify the list of capacities to be evaluated. If you leave the list blank, all capacities will be evaluated.
1. Press the "Run all" button to execute all cells in the notebook
1. Review the list of Semantic Models using the Large Semantic Model Storage Format found across your tenant
1. Review and apply relevant remediation suggestions listed at the bottom of this notebook.



In [ ]:
#Initialize the list of relevant capacities 
capacities = []

#If desired, specify capacities that you are interested in reviewing, by modifying the example below.
#If the capacities list is left empty, this notebook will analyze all capacities across the tenant
#capacities = [
#    'f8d9f93e-7fbe-4f7a-aed2-ca32bef55467',
#    '24345ed5-eeb2-4236-AF78-a66cb3f60242'
#    ]

In [ ]:
# Import relevant libraries
import sempy, json, time
import sempy.fabric as fabric
import pandas as pd

In [ ]:
#Convert list of capacities to lower case
capacities = [x.lower() for x in capacities]

#Get a list of workspaces
df = fabric.list_workspaces()

#Make sure that the workspaces are on a Fabric/Premium capacity
df = df[df['Is On Dedicated Capacity']]

#If a list of capacities was specified ensure that the workspace is on the capacity of interest
if capacities != []:
    df = df[df['Capacity Id'].isin(capacities)]

#Convert a list of workspace Ids to list
workspaces = df.Id.to_list()

#Define a function to initiate a metadata scan of relevant workspaces
def scanWorkspaces(workspaces):
    #Add the list of workspaces to the body of the request payload and submit a Post request to the scanner API endpoint
    body = {}
    body['workspaces'] = workspaces

    client = fabric.PowerBIRestClient()
    response = client.post(f"/v1.0/myorg/admin/workspaces/getInfo", json = body)

    #Get scanId
    return response.json()['id']

#Get scanId
scanId = scanWorkspaces(workspaces)
print('Metadata Scan Id: ' + scanId)

In [ ]:
#Define a function to get scan status
def getScanStatus(scanId):
    client = fabric.PowerBIRestClient()
    uri = f"/v1.0/myorg/admin/workspaces/scanStatus/" + scanId
    status = client.get(uri).json()['status']
    if status != 'Succeeded':
        raise ValueError("The status is not a success")
    else: 
        return status

#Define a function to call another function with an exponential backoff
def callWithBackoff(function, args=None, kwargs=None, maxRetries = 2, initialDelaySeconds = 1):
    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}
    i = 0
    while True:
        try:
            return function(*args, **kwargs)
        except:
            if i == maxRetries:
                raise TimeoutError("Maximum number of retries has been exceeded! For large Fabric tenants, consider increasing the number of retries!")
    delay = (initialDelaySeconds * pow(2, i))
    time.sleep(delay)
    i += 1

#Construct the mapping of arguments
scanIdKwargs = {}
scanIdKwargs["scanId"] = scanId

#Get the Scan status while implementing the exponential backoff
response = callWithBackoff(getScanStatus, args=None, kwargs = scanIdKwargs, maxRetries = 8, initialDelaySeconds = 5)
print('Metadata Scan Status: ' + response)

In [ ]:
#Define a function to retrieve scan results for a completed scan
def getScanResults(scanId):
    client = fabric.PowerBIRestClient()
    response = client.get(f"/v1.0/myorg/admin/workspaces/scanResult/" + scanId)
    return response.json()

#Get scan results
scanResults = getScanResults(scanId)
print('Scan results retrieved')

#### The following semantic models use Large Semantic Model Storage Format

In [ ]:
#Construct a Pandas data frame composed of the flattened JSON document
df = pd.json_normalize(scanResults, record_path = ['workspaces', 'datasets'], meta=[['workspace','capacityId'],['workspace', 'id'],['workspace', 'name']]) 

# Filter semantic models where 'targetStorageMode' == 'PremiumFiles'
df = df[df['targetStorageMode'] == "PremiumFiles"]

print('Found ' + str(df.shape[0]) + ' semantic models that use the Large Semantic Model Storage Format')

# Select relevant columns
df[['workspace.capacityId', 'workspace.id', 'workspace.name', 'id', 'name', 'targetStorageMode', 'createdDate', 'configuredBy']]

#### Remediation Suggestions
If you determine that you have semantic models in the Large Semantic Model Storage Format that you would like to move to a different region, you have several remediation options. The following options are listed in the order of complexity -- consider using the simplest option that would be sufficient to address your requirements. These suggestions are not intended to serve as a detailed step-by-step migration guide, but rather offer high-level ideas on how to execute the migration.

##### Option 1: Switch to small semantic model storage format
Check the size of the semantic model. If the semantic model is less than 10GB in size, you should be able to switch the semantic model to the "Small Semantic Model Storage Format" in preparation for the migration to another region.  Once the workspace containing your semantic model has been reassigned to a capacity in another region, you should be able to switch the semantic model back to the Large Semantic Model storage format in order to take advantage of performance benefits offered by this format.

```
Note: semantic models larger than 10GB cannot be switched to the small semantic model storage format.
```

##### Option 2: Redeploy semantic model as a new item
If your semantic model is too large to allow you to use Option 1, you may deploy your semantic model to the destination workspace in another region (as if it were a new model). Once the semantic model has been re-deployed you may need to refresh the semantic model to populate it with data, configure refresh schedules, configure permissions to match the configuration settings of your original semantic model. Similarly any reports or derived semantic models that depend on the original semantic model will need to be re-bound to the newly-deployed semantic model. Once your new semantic model has been deployed and properly configured, you may delete the original semantic model from the source workspace.

```
Note: for heavily-used semantic models with complex permissions and extensive downstream dependencies, re-deploying the semantic model will require additional effort.
```

##### Option 3: Shrink semantic model and switch to small semantic model storage format
If re-deploying your semantic model as outlined in Option 2 is undesirable, you may consider shrinking the size of the semantic model by performing a refresh operation that clears values from your semantic model. More specifically, you will execute the "clearValues" refresh. This refresh type can be performed over the XMLA endpoint (https://learn.microsoft.com/en-us/analysis-services/tmsl/refresh-command-tmsl?view=asallproducts-allversions#:~:text=all%20its%20dependents.-,clearValues,-Database%2C%0ATable) or using the advanced refresh REST API (https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/refresh-dataset#datasetrefreshtype). 

Once the clearValues refresh has been performed, the semantic model can be switched to the small semantic model storage format and the workspace containing the model can be moved to a capacity in the desired region. After the move, the semantic model can be switched back to the "large semantic model storage format" and refreshed to populate it with data.

```
Caution: performing the clearValues refresh will result in the deletion of all data from the semantic model -- therefore, please be sure that you have access to source data and can "rehydrate" the semantic model by refreshing it after the workspace containing the model has been moved to the target region.
```

##### Option 4: Backup and restore your semantic model
If Option 3 is not viable (for instance, if the historical source data that has been imported into the semantic model as part of an incremental refresh process is no longer available), you may consider the following methodology to complete the refresh process.
1. [Backup the semantic mode to ADLS Gen2l](https://learn.microsoft.com/en-us/power-bi/enterprise/service-premium-backup-restore-dataset)
1. Restore the semantic model from backup to a workspace hosted on a capacity in the desired region.

```
Note: Once the semantic model has been restored, you will need to complete additional configuration steps, such as configure refresh schedules, configure permissions to match the configuration settings of your original semantic model. Similarly any reports or derived semantic models that depend on the original semantic model will need to be re-bound to the newly-deployed semantic model.
```
##### Option 5: Backup, migrate and restore your semantic model
Finally, you may consider a hybrid strategy that combines Option 3 and Option 4. While this approach involves the greatest number of steps, it allows you to preserve historical data as well as the configuration settings of the existing semantic model that needs to be migrated to a workspace on a capacity in another region.
1. [Backup the semantic mode to ADLS Gen2l](https://learn.microsoft.com/en-us/power-bi/enterprise/service-premium-backup-restore-dataset)
1. Shrink the semantic model by performing the clearValues refresh (as described in Option 3)
1. Switch the semantic model to the small semantic model storage format
1. Move the semantic model to the desired region (by assigning the workspace hosting the model to a suitable capacity in the target region)
1. Switch the semantic model back to the large semantic model storage format
1. Restore the semantic model from backup (by overwriting the existing model that has just been migrated)

#### Conclusion
After reviewing the list of semantic models that use the Large Semantic Model Storage Format, select an optimal remediation option for each of the affected semantic models. Consider using the simplest option that would be sufficient to address your requirements. Finally, complete the migration of your workspaces to a capacity in a different region.